In [2]:
import pandas as pd
import numpy as np
import os

# Read data 

In [3]:
dir_usda = r"D:\US_CornSoy_ClassificationYield\TabularData"
usda_area = pd.read_csv(os.path.join(dir_usda, "US_County_CornSoy_Area.csv"))

In [25]:
usda_area.head(2)

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2023,YEAR,NaN,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,"11,500,000",NaN
1,SURVEY,2023,YEAR,NaN,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,SOYBEANS,SOYBEANS - ACRES PLANTED,TOTAL,NOT SPECIFIED,"10,000,000",NaN


In [26]:
usda_area["Data Item"].unique()

array(['CORN - ACRES PLANTED', 'SOYBEANS - ACRES PLANTED'], dtype=object)

In [ ]:
usda_area['Value']

In [4]:
state_area = usda_area[usda_area['Geo Level'] == 'STATE']
cnty_area = usda_area[usda_area['Geo Level'] == 'COUNTY']
#SUBSET COLUMNS
state_area = state_area[['Year', 'State', 'State ANSI', 'Commodity', 'Value']]
cnty_area = cnty_area[['Year', 'State', 'State ANSI', 'County', 'County ANSI', 'Commodity', 'Value']]
#Rename Columns
state_area.columns = ['Year', 'State', 'StateANSI', 'Crop', 'AreaAcres_usda']
cnty_area.columns = ['Year', 'State', 'StateANSI', 'County', 'CntyANSI', 'Crop', 'AreaAcres_usda']
#remove rows whose CntyANSI column is NaN
cnty_area = cnty_area.dropna(subset=['CntyANSI']) 
#convert CntyANSI to int with no decimal, and fill with leading zeros to make 3 digits
cnty_area['CntyANSI'] = cnty_area['CntyANSI'].astype(int).astype(str).str.zfill(3)
#convert State and County and Crop column to be uppercase for only the first letter
cnty_area['State'] = cnty_area['State'].str.title()
cnty_area['County'] = cnty_area['County'].str.title()
cnty_area['Crop'] = cnty_area['Crop'].str.title()
state_area['State'] = state_area['State'].str.title()
state_area['Crop'] = state_area['Crop'].str.title()
#remove comma in AreaAcres_usda column, and convert AreaAcres_usda to float
cnty_area['AreaAcres_usda'] = cnty_area['AreaAcres_usda'].str.replace(',', '').astype(float)
state_area['AreaAcres_usda'] = state_area['AreaAcres_usda'].str.replace(',', '').astype(float)
#convert Soybeans to Soybean
cnty_area['Crop'] = cnty_area['Crop'].str.replace('Soybeans', 'Soybean')
state_area['Crop'] = state_area['Crop'].str.replace('Soybeans', 'Soybean')

# Calculate area ratio

In [9]:
state_area.head()

,Year,State,StateANSI,Crop,AreaAcres_usda
0,2023,Illinois,17,Corn,11500000.0
1,2023,Illinois,17,Soybean,10000000.0
2,2023,Indiana,18,Corn,5500000.0
3,2023,Indiana,18,Soybean,5500000.0
4,2023,Iowa,19,Corn,13400000.0


In [11]:
state_area.shape

(696, 5)

In [15]:
print(state_area['State'].unique())
print(state_area['Crop'].unique())
print(state_area['Year'].unique())

['Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Michigan' 'Minnesota'
 'Missouri' 'Nebraska' 'Ohio' 'South Dakota' 'Wisconsin']
['Corn' 'Soybean']
[2023 2022 2021 2020 2019 2018 2017 2016]


In [13]:
state_area.tail()

,Year,State,StateANSI,Crop,AreaAcres_usda
12842,2016,Ohio,39,Soybean,4650000.0
12843,2016,South Dakota,46,Corn,5700000.0
12844,2016,South Dakota,46,Soybean,5000000.0
12845,2016,Wisconsin,55,Corn,4000000.0
12846,2016,Wisconsin,55,Soybean,1950000.0


In [21]:
for state in state_area['State'].unique():
    for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]:
        corn_area = state_area[(state_area['State'] == state) & (state_area['Crop'] == 'Corn') & (state_area['Year'] == year)]['AreaAcres_usda'].values
        soybean_area = state_area[(state_area['State'] == state) & (state_area['Crop'] == 'Soybean')& (state_area['Year'] == year)]['AreaAcres_usda'].values
        cornsoy_ratio = corn_area / soybean_area
        print(state, year, cornsoy_ratio)

Illinois 2016 [1.14851485 1.21      ]
Illinois 2017 [1.05660377 1.10784314]
Illinois 2018 [1.01851852 1.00917431 1.03773585 1.01851852]
Illinois 2019 [1.05527638 1.07       1.06796117 1.06666667 1.05      ]
Illinois 2020 [1.09708738 1.04807692 1.07619048 1.10679612]
Illinois 2021 [1.03773585 1.04672897 1.01869159 1.03773585 1.03773585]
Illinois 2022 [1.         0.95535714 0.97272727 1.        ]
Illinois 2023 [1.15       1.15       1.01851852]
Indiana 2016 [0.99115044 1.04504505]
Indiana 2017 [0.89915966 0.93333333]
Indiana 2018 [0.88333333 0.82258065 0.83606557 0.89915966]
Indiana 2019 [0.92592593 0.94444444 1.03773585 0.96491228 0.94444444]
Indiana 2020 [0.93913043 0.94736842 1.07407407 0.94736842]
Indiana 2021 [0.95575221 0.94736842 0.89655172 0.94736842 0.94736842]
Indiana 2022 [0.8974359  0.87179487 0.86440678 0.88888889]
Indiana 2023 [1.         1.         0.98214286]
Iowa 2016 [1.46315789 1.43298969]
Iowa 2017 [1.33       1.31683168]
Iowa 2018 [1.32663317 1.34343434 1.35714286 1.

In [22]:
corn_area

array([4000000., 4000000., 3950000.])